In [1]:
import joblib
import pandas as pd

from rdkit import Chem
from sklearn.metrics import roc_auc_score
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F

import numpy as np
from rdkit.Chem import rdFingerprintGenerator
import deepchem as dc
import random
import optuna

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    log_loss,
    roc_auc_score,
    precision_score,
    recall_score,
    confusion_matrix,
)


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/m12gbs1/miniconda3/envs/dili/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Load pre-trained models

In [3]:
# load the results back
loaded_results = joblib.load("models/tpot_results.joblib")

# Convert results to DataFrame
tpot_df = pd.DataFrame(loaded_results)
tpot_df.head()

,Best model,PCA Components,Model Name,Parameters,AUC,Precision,Recall,Sensitivity,Specificity
0,"((ExtraTreeClassifier(max_features=0.55, min_s...",10,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.860355,0.911765,0.841629,0.841629,0.723077
1,"((ExtraTreeClassifier(criterion='entropy', max...",20,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.901636,0.930348,0.846154,0.846154,0.784615
2,"((ExtraTreeClassifier(criterion='entropy', max...",50,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.899617,0.919811,0.882353,0.882353,0.738462
3,"(MinMaxScaler(), (ExtraTreeClassifier(max_feat...",100,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.916881,0.920930,0.895928,0.895928,0.738462
4,"(KNeighborsClassifier(n_neighbors=4, weights='...",200,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.899060,0.938144,0.823529,0.823529,0.815385


In [4]:
model1 = loaded_results[3]["Best model"]
model1

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('extratreesclassifier',
                 ExtraTreesClassifier(max_features=0.35000000000000003,
                                      min_samples_leaf=3, min_samples_split=13,
                                      random_state=42))])

In [5]:
class GCN(torch.nn.Module):
    def __init__(
        self,
        num_node_features,
        num_classes,
        num_layers=3,
        hidden_dim=64,
        dropout_prob=0.5,
        activation="relu",
    ):
        super(GCN, self).__init__()

        # Store activation function dynamically
        if activation == "relu":
            self.activation = F.relu
        elif activation == "tanh":
            self.activation = F.tanh
        else:
            raise ValueError("Unsupported activation function")

        self.dropout_prob = dropout_prob

        # Dynamically define the GCN layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(num_node_features, hidden_dim))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))

        # Final fully connected layer
        self.fc = torch.nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Apply GCN layers dynamically
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.activation(x)

        # Global pooling (combine different pooling methods)
        x = torch.cat([global_mean_pool(x, batch), global_add_pool(x, batch)], dim=1)

        # Apply dropout
        x = F.dropout(x, p=self.dropout_prob, training=self.training)

        # Final classification layer
        return F.log_softmax(self.fc(x), dim=1)


# load model from pth
best_params = {
    "hidden_dim": 196,
    "dropout_prob": 0.5290715263519528,
    "lr": 0.0006244887574274183,
    "weight_decay": 8.693717191600785e-06,
    "num_layers": 5,
    "activation": "relu",
}
model2 = GCN(
    num_node_features=70,
    num_classes=2,
    num_layers=best_params["num_layers"],
    hidden_dim=best_params["hidden_dim"],
    dropout_prob=best_params["dropout_prob"],
    activation=best_params["activation"],
).to(device)

model2.load_state_dict(torch.load("models/graph_gcn_x25.pth"))


/tmp/ipykernel_780005/989027252.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load("models/graph_gcn_x25.pth"))


<All keys matched successfully>

# Data preparation

## PCA prep

In [6]:
# Load pd_train
pd_train = pd.read_parquet("data/training_class.parquet")
pd_train["label"] = pd_train["Class"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0)
print(pd_train.shape)

(1241, 16094)


In [7]:
# Load pd_test
pd_test = pd.read_parquet("data/testing_class.parquet")
pd_test["label"] = pd_test["Class"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0)
print(pd_test.shape)

(286, 16094)


In [8]:
X_train = pd_train.drop(columns=["Class", "label"])
y_train = pd_train["label"]

X_test = pd_test.drop(columns=["Class", "label"])
y_test = pd_test["label"]

In [9]:
# get X PCA components 100
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

## Graph prep

In [10]:
# Featurization using DeepChem's MolGraphConvFeaturizer
from utils.SmilesEnumeration import SmilesEnumerator


def featurize_smiles(smiles):
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
    graph_data = featurizer.featurize([smiles])[0]

    # Get DeepChem atom features
    atom_features_deepchem = graph_data.node_features

    return atom_features_deepchem


# Function to generate Morgan Fingerprints (ECFP)
def generate_ecfp(smiles):
    # Morgan fingerprint generator
    mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=4096)

    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    return mfgen.GetFingerprintAsNumPy(molecule)


# Function to convert SMILES to PyTorch Geometric Data object using DeepChem featurizer
def smiles_to_graph_featurizer(smiles):
    # Featurization using DeepChem
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

    # Featurize the SMILES string using DeepChem
    graph_data = featurizer.featurize([smiles])[0]
    return graph_data.node_features, graph_data.edge_features, graph_data.edge_index


# Function to extract atom features
def atom_features(atom, ecfp):
    # Get the atom index for corresponding ECFP value
    atom_idx = atom.GetIdx()

    return [
        atom.GetAtomicNum(),  # Atomic number
        atom.GetDegree(),  # Number of bonds
        atom.GetTotalNumHs(),  # Total number of hydrogens
        atom.GetFormalCharge(),  # Formal charge of the atom
        atom.GetImplicitValence(),  # Implicit valence
        atom.GetNumRadicalElectrons(),  # Number of radical electrons
        int(atom.GetIsAromatic()),  # Is the atom aromatic?
        atom.GetMass(),  # Atomic mass
        atom.GetHybridization().real,  # Hybridization state (SP, SP2, SP3, etc.)
        ecfp[atom_idx],  # Morgan fingerprint (ECFP) for the atom
    ]


# Function to extract bond features
def bond_features(bond):
    bond_type = bond.GetBondTypeAsDouble()  # Bond type as a float
    is_aromatic = bond.GetIsAromatic()  # Aromatic bond
    is_conjugated = bond.GetIsConjugated()  # Conjugated bond
    is_in_ring = bond.IsInRing()  # Whether the bond is part of a ring
    stereo = bond.GetStereo()  # Bond stereochemistry

    # Convert stereo information to a one-hot encoded format
    stereo_one_hot = [0, 0, 0, 0]  # Stereo options: None, E, Z, Other
    if stereo == Chem.BondStereo.STEREONONE:
        stereo_one_hot[0] = 1
    elif stereo == Chem.BondStereo.STEREOE:
        stereo_one_hot[1] = 1
    elif stereo == Chem.BondStereo.STEREOZ:
        stereo_one_hot[2] = 1
    else:
        stereo_one_hot[3] = 1

    # Combine all features into a single tensor
    return [
        bond_type,
        float(is_aromatic),
        float(is_conjugated),
        float(is_in_ring),
    ] + stereo_one_hot


# Convert SMILES to PyTorch Geometric Data object
def smiles_to_graph(smiles, label):
    mol = Chem.MolFromSmiles(smiles)

    atom_features_list = []
    edge_index = []
    edge_attr = []

    # DeepChem features
    atom_features_deepchem = featurize_smiles(smiles)

    # Generate Morgan Fingerprint (ECFP)
    ecfp_features = generate_ecfp(smiles)

    # Generate Molecule Graph Convolution features
    mol_graph_node_features, mol_graph_edge_features, mol_graph_edge_index = (
        smiles_to_graph_featurizer(smiles)
    )

    # Nodes (atoms)
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_features(atom, ecfp_features))

    atom_features_list = np.array(atom_features_list)

    # Edges (bonds)
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        # Append bidirectional edges for undirected graphs
        edge_index.append([i, j])
        edge_index.append([j, i])

        # Append bond features for both directions
        edge_attr.append(bond_features(bond))
        edge_attr.append(bond_features(bond))

    # Convert atom features to a tensor
    combined_features = np.concatenate(
        (atom_features_list, atom_features_deepchem, mol_graph_node_features), axis=1
    )
    x = torch.tensor(combined_features, dtype=torch.float)

    # Convert edge indices and features to tensors, handle empty edge case
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # combine edge features from ECFP and MolGraphConv
    edge_attr = np.array(edge_attr)
    edge_attr = np.concatenate((edge_attr, mol_graph_edge_features), axis=1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # Label (target)
    y = torch.tensor([label], dtype=torch.long)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)


# Function to load data from CSV and apply SMILES augmentation for training
def load_data_from_csv(file_path, augment=False, augment_times=1):
    df = pd.read_csv(file_path)
    smiles_list = df["Smiles"].values
    labels = df["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0).values

    data_list = []

    # Initialize the SmilesEnumerator for data augmentation
    for smiles, label in zip(smiles_list, labels):
        # For test data, no augmentation, just use canonical SMILES
        graph_data = smiles_to_graph(smiles, label)
        data_list.append(graph_data)

    data_list = []

    # Initialize the SmilesEnumerator for data augmentation
    sme = SmilesEnumerator()
    for smiles, label in zip(smiles_list, labels):
        if augment:
            # Apply SMILES randomization (augmentation) for training data
            for _ in range(augment_times):
                augmented_smiles = sme.randomize_smiles(smiles)
                graph_data = smiles_to_graph(augmented_smiles, label)
                data_list.append(graph_data)
        else:
            # For test data, no augmentation, just use canonical SMILES
            graph_data = smiles_to_graph(smiles, label)
            data_list.append(graph_data)

    return data_list

In [11]:
# Load training and testing data
training_data = load_data_from_csv("data_smiles/Training_Group.csv")
testing_data = load_data_from_csv("data_smiles/Testing_Group.csv")

In [12]:
# Create data loaders
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=32, shuffle=False)

# Model training

In [19]:
tpot_preds = model1.predict_proba(X_train_pca)[:, 1]

# Predict on validation set
model2.eval()
gcn_preds = []
with torch.no_grad():
    for data in train_loader:
        data = data.to(device)
        out = model2(data)
        gcn_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

gcn_preds = np.array(gcn_preds)

# Combine predictions as stacked features
stacked_features_train = np.vstack([tpot_preds, gcn_preds]).T
stacked_features_train.shape

(1241, 2)

In [14]:
# Train meta-model
meta_model = LogisticRegression(random_state=42)
meta_model.fit(stacked_features_train, y_train)


LogisticRegression(random_state=42)

In [15]:
from sklearn.metrics import roc_auc_score, accuracy_score

# TPOT predictions on test set
tpot_test_preds = model1.predict_proba(X_test_pca)[:, 1]

# GCN predictions on test set
gcn_test_preds = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model2(data)
        gcn_test_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

# Stack test predictions
stacked_features_test = np.vstack([tpot_test_preds, gcn_test_preds]).T

# Make final predictions with meta-model
final_predictions = meta_model.predict(stacked_features_test)
final_auc = roc_auc_score(y_test, final_predictions)
final_accuracy = accuracy_score(y_test, final_predictions)

print("Stacked Model Test AUC:", final_auc)
print("Stacked Model Test Accuracy:", final_accuracy)


Stacked Model Test AUC: 0.8067873303167421
Stacked Model Test Accuracy: 0.8356643356643356


In [22]:
from tpot import TPOTClassifier

# Set up TPOT with basic configuration
tpot = TPOTClassifier(
    generations=100,
    population_size=100,
    verbosity=2,
    scoring="f1",  # AUC
    random_state=42,
    n_jobs=16,  # Use 4 cores
)

# Fit TPOT on the PCA-transformed data
tpot.fit(stacked_features_train, y_train)

# Access the best pipeline directly after fitting
best_pipeline = tpot.fitted_pipeline_

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.972892670124548

Generation 2 - Current best internal CV score: 0.972892670124548

Generation 3 - Current best internal CV score: 0.9736089714052077

Generation 4 - Current best internal CV score: 0.9736089714052077

Generation 5 - Current best internal CV score: 0.9736089714052077

Generation 6 - Current best internal CV score: 0.9736089714052077

Generation 7 - Current best internal CV score: 0.9736089714052077

Generation 8 - Current best internal CV score: 0.9736897825107544

Generation 9 - Current best internal CV score: 0.9736897825107544

Generation 10 - Current best internal CV score: 0.9736897825107544

Generation 11 - Current best internal CV score: 0.9736897825107544

Generation 12 - Current best internal CV score: 0.9736897825107544

Generation 13 - Current best internal CV score: 0.974433078144561

Generation 14 - Current best internal CV score: 0.974433078144561

Generation 15 - Current best internal CV score: 0.9744330781

In [21]:
# Check if the model has predict_proba
from pprint import pprint


if hasattr(best_pipeline, "predict_proba"):
    y_pred = best_pipeline.predict_proba(stacked_features_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)
else:
    # Fallback to accuracy if predict_proba is unavailable
    y_pred = best_pipeline.predict(stacked_features_test)
    auc = accuracy_score(
        y_test, y_pred
    )  # Substitute with accuracy if AUC can't be calculated

y_pred = (y_pred > 0.5).astype(int)  # Binarize predictions for threshold of 0.5
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)  # Same as sensitivity for positive class

# Calculate sensitivity and specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Extract model name and parameters
model_name = type(
    best_pipeline.steps[-1][1]
).__name__  # Name of the final estimator
model_params = best_pipeline.steps[-1][
    1
].get_params()  # Params of the final estimator

# Append results to list
result = {
    "Best model": best_pipeline,
    "Model Name": model_name,
    "Parameters": model_params,
    "AUC": auc,
    "Precision": precision,
    "Recall": recall,
    "Sensitivity": sensitivity,
    "Specificity": specificity,
}

pprint(result)

{'AUC': 0.9042116254785938,
 'Best model': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('stackingestimator',
                 StackingEstimator(estimator=GaussianNB())),
                ('sgdclassifier',
                 SGDClassifier(alpha=0.01, eta0=0.01, fit_intercept=False,
                               l1_ratio=1.0, learning_rate='invscaling',
                               loss='modified_huber', penalty='elasticnet',
                               random_state=42))]),
 'Model Name': 'SGDClassifier',
 'Parameters': {'alpha': 0.01,
                'average': False,
                'class_weight': None,
                'early_stopping': False,
                'epsilon': 0.1,
                'eta0': 0.01,
                'fit_intercept': False,
                'l1_ratio': 1.0,
                'learning_rate': 'invscaling',
                'loss': 'modified_huber',
                'max_iter': 1000,
                'n_iter_no_change': 5,
                'n_jo